In [ ]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

import json
from sqlalchemy import create_engine, inspect, Table, Column
import pandas as pd
from ilxutils.args_reader import read_args
from ilxutils.scicrunch_client import scicrunch
import sys
import time
from ilxutils.interlex_sql import interlex_sql
import re
from collections import defaultdict

args = read_args(api_key='../keys/production_api_scicrunch_key.txt', db_url='../keys/production_engine_scicrunch_key.txt', production=True)
sql = interlex_sql(engine_key=args.db_url)
sci = scicrunch(api_key=args.api_key, base_path=args.base_path, engine_key=args.db_url)

In [ ]:
labels_to_ids = sql.get_labels_to_ids_dict()

In [ ]:
tier1 = '/home/troy/Desktop/NDA/nda-cdes-first-tier-only-update-ready.json'
tier2 = '/home/troy/Desktop/NDA/complete_nda_tier2_production_ref.json'
nda_names = list(json.load(open(tier1)).keys()) + list(json.load(open(tier2)).keys())
tier1_data = json.load(open(tier1))
tier2_data = json.load(open(tier2))

In [ ]:
list(tier2_data)[0]


In [ ]:
tier2_data[list(tier2_data)[0]][0]

In [ ]:
def get_id(pad):
    for value in pad['annotations']:
        if 'id : ' in value.get('value'):
            return value.get('value').split(':')[-1].strip()

data = []
iri = 'http://uri.interlex.org/NDA/uris/datadictionary/elements/'
prefix = 'NDA.CDE:'
for x in tier2_data.items():
    for y in x[1]:
        tid = get_id(y)
        #print(x[0], y['label'], y['id'])
        data.append({'id':y['id'], 'existing_ids':{'iri':iri+tid, 'curie':prefix+tid, 'change':False, 'delete':False}})
data[0]

In [ ]:
json.dump(data, open('/home/troy/Desktop/nda-existing-ids-update.json', 'w'), indent=4)
#python3 /home/troy/Desktop/interlexutils/interlexutils/api_wrapper.py updateTerms -f /home/troy/Desktop/nda-existing-ids-update.json -p

In [ ]:
annotations=sql.get_annotations()

In [ ]:
terms = sql.get_terms()

In [ ]:
exids = sql.get_existing_ids()

In [ ]:
exids.loc[exids.iri.str.contains('http://uri.interlex.org/NDA/uris/datadictionary/elements/')]